# Customer Churn Prediction Model


This project aims to leverage historical customer data and machine learning algorithms to build an accurate churn prediction model.  
By identifying customers likely to churn, companies can take targeted actions to address their concerns,   
offer tailored incentives, or improve their overall experience,   
ultimately reducing customer attrition and its associated costs.

## Data

This dataset contains customer information and attributes relevant to assessing customer churn, which refers to the likelihood of customers leaving a service or company. The dataset's primary objective is to provide insights into customer behavior and help determine whether a customer will churn or not. Each row in the dataset represents a customer with specific attributes, and the "Churn" column indicates whether the customer has churned (1) or not (0).

Dataset Attributes:
- **CustomerId**: The unique identifier for each customer.
- **Surname**: The last name of the customer.
- **CreditScore**: The credit score of the customer.
- **Geography**: The geographical location of the customer.
- **Gender**: The gender of the customer.
- **Age**: The age of the customer.
- **Tenure**: The number of years the customer has been with the company.
- **Balance**: The account balance of the customer.
- **NumOfProducts**: The number of products the customer has with the company.
- **IsActiveMember**: Whether the customer is an active member (1) or not (0).
- **Churn**: Whether the customer has churned (1) or not (0).

## Machine Learning Task:
This dataset is suitable for a supervised binary classification task, where machine learning models   
can be trained to predict customer churn based on the provided attributes.   
The models aim to classify customers as churned (1) or not churned (0).


## Exploratory Data Analysis (EDA)
EDA is a crucial aspect of data science that helps in understanding the underlying structure of the data. It involves analyzing and summarizing data visually and statistically to uncover patterns or relationships. The main goals of EDA include:

## Gaining a deeper understanding of the data
- Identifying data quality issues
- Developing initial insights and hypotheses
- Selecting features for modeling or further analysis

In [ ]:
# import modules
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split